In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization, Dense, Dropout, Flatten
from tensorflow.keras import regularizers

In [2]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from keras.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [27]:
train_directory=r"D:\waste_archive\garbage_classification\train"
validation_directory=r"D:\waste_archive\garbage_classification\test"

In [28]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values to [0, 1]
    rotation_range=15  # Random rotation 180,15
    #width_shihttps://colab.research.google.com/drive/1mqChz1NqFPFeWYCIPJDfWsvJHdyP37dj?authuser=3#scrollTo=KFOIhgnCzIZdft_range=0.2  # Random horizontal shift
    #height_shift_range=0.2  # Random vertical shift
    #shear_range=0.2,  # Shearing dont do
    #zoom_range=0.2  # Zooming
    #horizontal_flip=True  # Random horizontal flip
    #fill_mode="nearest"  # Filling in newly created pixels
)

In [29]:
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [30]:
train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(64, 64),  # Update with your desired image size
    batch_size=26,
    class_mode="categorical" # Use "categorical" for multiclass classification
)

Found 12409 images belonging to 12 classes.


In [31]:
validation_generator = validation_datagen.flow_from_directory(
    validation_directory,
    target_size=(64, 64),
    batch_size=26,
    class_mode="categorical"
)


Found 3106 images belonging to 12 classes.


In [81]:
model = Sequential()


model.add(BatchNormalization(input_shape=(64, 64, 3)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))

model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.4))


model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(Dense(12, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [82]:
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator
)


Epoch 1/5
478/478 [==============================] - 351s 728ms/step - loss: 0.3298 - accuracy: 0.4676 - val_loss: 0.1626 - val_accuracy: 0.5911
Epoch 2/5
478/478 [==============================] - 175s 366ms/step - loss: 0.1650 - accuracy: 0.6167 - val_loss: 0.1475 - val_accuracy: 0.6590
Epoch 3/5
478/478 [==============================] - 134s 280ms/step - loss: 0.1508 - accuracy: 0.6521 - val_loss: 0.1370 - val_accuracy: 0.6629
Epoch 4/5
478/478 [==============================] - 125s 261ms/step - loss: 0.1409 - accuracy: 0.6810 - val_loss: 0.1305 - val_accuracy: 0.6835
Epoch 5/5
478/478 [==============================] - 127s 266ms/step - loss: 0.1329 - accuracy: 0.7069 - val_loss: 0.1167 - val_accuracy: 0.7283


In [83]:
# Make predictions on the test set
y_pred = model.predict_generator(validation_generator)

# Obtain true labels from the validation generator
y_true_classes = validation_generator.classes

# If you have more than two classes and are using softmax activation, use:
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert probabilities to binary predictions for a binary classification problem
y_pred_classes = np.round(y_pred)

C:\Users\Nidhi\AppData\Local\Temp\ipykernel_32640\1548223628.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_pred = model.predict_generator(validation_generator)


In [84]:
y_pred_classes

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [94]:
model.save("waste_model_binary.h5")

In [85]:
import cv2
import numpy as np
import tensorflow as tf

# Load your pre-trained model
#model = tf.keras.models.load_model(r"D:\waste_model.h5")

# Function to preprocess the image
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (image_width, image_height))
    img = img / 255.0  # Normalize pixel values to [0, 1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Define the size of images expected by your model
image_width = 64
image_height = 64

# Function to make predictions
def predict(image_path):
    # Preprocess the image
    img = preprocess_image(image_path)
    # Make prediction
    prediction = model.predict(img)
    return prediction

# Path to your image
image_path = r"C:\Users\Nidhi\Downloads\segregated-garbage-glass-metal-paper-and-plastic-fj4t1c.jpg"

# Make prediction
prediction = predict(image_path)

# Assuming your model predicts classes and you have a list of class labels

class_labels = ['battery',
 'biological',
 'brown-glass',
 'cardboard',
 'clothes',
 'green-glass',
 'metal',
 'paper',
 'plastic',
 'shoes',
 'trash',
 'white-glass']
# Get the index of the predicted class with highest probability
predicted_class_index = np.argmax(prediction)
predicted_class_label = class_labels[predicted_class_index]

print("Predicted Class:", predicted_class_label)
#print("Prediction Probabilities:", y_pred_classes)


1/1 [==============================] - 0s 300ms/step
Predicted Class: paper


In [92]:
threshold = 0.05  # Adjust as needed
predicted_classes = (prediction > threshold).astype(int)


In [93]:
for i, class_label in enumerate(class_labels):
    if predicted_classes[0, i] == 1:
        print("Predicted Class:", class_label)

Predicted Class: metal
Predicted Class: paper
Predicted Class: plastic
Predicted Class: white-glass
